In [ ]:
import json
from IPython.display import clear_output, HTML, display
import ipywidgets.widgets as widgets
from tqdm.notebook import tqdm

import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')

import ballsbot.config as config
config.ENABLE_MULTIPROCESSING = False

In [ ]:
new_format = True
with open('track_2024-08-20.json', 'r') as hf:
    if new_format:
        track = []
        for line in hf:
            try:
                frame = json.loads(line)
                track.append(frame)
            except Exception as e:
                print(f'broken frame {e}')
    else:
        track = json.loads(hf.read())

In [ ]:
print(len(track))

In [ ]:
# first_i = 1
# second_i = 5

first_i = 248
second_i = 250

# first_i = 101
# second_i = 102

# first_i = 120
# second_i = 121

# first_i = 121
# second_i = 122

first_frame = track[first_i]
second_frame = track[second_i]

print(first_frame['tracker']['current_pose'])
print(second_frame['tracker']['current_pose'])

In [ ]:
import ballsbot.drawing as drawing
from ballsbot.lidar import calibration_to_xywh
from ballsbot.drawing import BotPoseAbsCoordsDrawing
from ballsbot.utils import keep_rps
from ballsbot.lidar import revert_transformation_to_cloud
from ballsbot.dashboard import Dashboard

In [ ]:
dashboard = Dashboard(widgets, image_size=800)

augmented_painter = BotPoseAbsCoordsDrawing(dashboard, 'augmented')
dashboard.add_subplot('augmented', augmented_painter.get_drawing_func_name())
sparse_painter = BotPoseAbsCoordsDrawing(dashboard, 'sparse')
dashboard.add_subplot('sparse', sparse_painter.get_drawing_func_name())

display(dashboard.get_image())

In [ ]:
only_nearby_meters = 4
calibration = {
    'angle_fix': 3.0810609316737434,  # lidar orientation (radians)
    'fl_x': 0.2041686441507201,  # front left corner coords
    'fl_y': 0.10426277741236079,
    'rr_x': -0.08011094659163859,  # rear right corner coords
    'rr_y': -0.0988675829003773,
}
self_position = calibration_to_xywh(calibration)

for frame_index, track_frame, painter in ((first_i, first_frame, augmented_painter), (second_i, second_frame, sparse_painter)):
    pose = {'ts': track_frame['tracker']['current_pose']['ts'], 'x': 0., 'y': 0., 'teta': 0.}

    if track_frame['tracker']['free_tile_centers']:
        cells = [track_frame['tracker']['target_point']] + track_frame['tracker']['free_tile_centers']
        transformation = (track_frame['tracker']['current_pose']['x'], track_frame['tracker']['current_pose']['y'], track_frame['tracker']['current_pose']['teta'])
        transformed_cells = revert_transformation_to_cloud(cells, transformation)
        free_cells = transformed_cells[1:]
        target_point = transformed_cells[0]
    else:
        free_cells = None
        target_point = None

    painter.update_image(
        [pose],
        track_frame['lidar']['augmented_points'],
        self_position,
        free_cells=free_cells,
        target_point=target_point,
        image_text=str(frame_index),
        only_nearby_meters=only_nearby_meters
    )
        
dashboard.redraw()

In [ ]:
# sys.path.append('/usr/local/lib/python3.6/dist-packages/python_pcl_ndt-0.2-py3.6-linux-x86_64.egg')

In [ ]:
raw_result = [
    second_frame['tracker']['current_pose'][k] - first_frame['tracker']['current_pose'][k]
    for field_i, k in enumerate(['x', 'y', 'teta'])
]
print(raw_result)

In [ ]:
diff_image = widgets.Image(format='jpeg', width=640, height=640)
display(diff_image)

In [ ]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from io import BytesIO

from ballsbot.utils import figsize_from_image_size
from ballsbot.lidar import apply_transformation_to_cloud, revert_transformation_to_cloud

def get_dashboard_image(image, lidar_points1, lidar_points2, plot_size=1.0):
    figsize = figsize_from_image_size(image)
    fig = Figure(figsize=figsize)
    canvas = FigureCanvas(fig)

    ax = fig.add_subplot(plot_size, plot_size, 1)
    ax.set_xlim(-only_nearby_meters, only_nearby_meters)
    ax.set_ylim(-only_nearby_meters, only_nearby_meters)

    lidar_x_points = [x[0] for x in lidar_points1]
    lidar_y_points = [x[1] for x in lidar_points1]
    ax.scatter(lidar_x_points, lidar_y_points, marker='o', s=5, c='blue')

    lidar_x_points = [x[0] for x in lidar_points2]
    lidar_y_points = [x[1] for x in lidar_points2]
    ax.scatter(lidar_x_points, lidar_y_points, marker='o', s=5, c='red')

    ax.grid(which='both', linestyle='--', alpha=0.5)

    canvas.draw()
    jpeg = BytesIO()
    canvas.print_jpg(jpeg)
    image.value = jpeg.getvalue()

get_dashboard_image(
    diff_image,
    first_frame['lidar']['augmented_points'],
    # second_frame['lidar']['augmented_points'],
    apply_transformation_to_cloud(second_frame['lidar']['augmented_points'], raw_result),
)

In [ ]:
from time import time
from python_pcl_ndt import python_pcl_ndt


class NDT:
    def __init__(self, grid_size=20., box_size=1.5, iterations_count=1000, optim_step=(0.1, 0.1, 0.01), eps=0.0001):
        self.grid_size = grid_size
        self.box_size = box_size
        self.iterations_count = iterations_count
        self.optim_step = optim_step
        self.eps = eps

    def get_optimal_transformation(self, cloud_one, cloud_two, start_point=(0., 0., 0.)):
        return python_pcl_ndt.get_transformation_vector_wrapper(
            cloud_one,
            cloud_two,
            self.iterations_count,  # iter
            self.box_size,  # grid_step
            self.grid_size,  # grid_extent
            self.optim_step[0],  # optim_step_x
            self.optim_step[1],  # optim_step_y
            self.optim_step[2],  # optim_step_theta
            self.eps,  # epsilon
            start_point[0],  # guess_x
            start_point[1],  # guess_y
            start_point[2],  # guess_theta
        )

def _try_converge(current_points, previous_points, start_point, **kwargs):
    ndt = NDT(**kwargs)
    return ndt.get_optimal_transformation(
        previous_points,
        current_points,
        start_point=start_point,
    )

def fix_pose(current_points, previous_points, raw_result):
    """
    :param current_points - [[x1, y1], ...] from lidar
    :param previous_points - [[x1, y1], ...] from lidar
    :param raw_result - [dx_raw, dy_raw, dteta_raw] poses diff from odometry
    """

    optim_step = (0.1, 0.1, 0.01)
    for start_point in (raw_result, [0., 0., 0.]):
        dx, dy, dteta, converged, score = _try_converge(current_points, previous_points, start_point, optim_step=optim_step)
        if converged == 1. and score < 0.02:
            return dx, dy, dteta
    for box_size in (1.5, 1.0):
        cur_result = raw_result
        for optim_step in ((0.1, 0.1, 0.02), (0.1, 0.1, 0.01)):
            for start_point in (cur_result, [0., 0., 0.]):
                dx, dy, dteta, converged, score = _try_converge(current_points, previous_points, start_point, optim_step=optim_step, box_size=box_size)
                if converged == 1. and score < 0.5:
                    break
            else:
                continue

            if score < 0.06:
                break
            else:
                # print(f'next({score}) {cur_result} = [{dx}, {dy}, {dteta}]')
                cur_result = [dx, dy, dteta]
        else:
            continue
        if score < 0.06:
            break
    else:
        if not converged or score > 0.39:
            return None
        # elif score > 0.2:
        #     print(f'wtf({converged}, {score})')

    return dx, dy, dteta

In [ ]:
print(raw_result)
fixed_result = fix_pose(second_frame['lidar']['augmented_points'], first_frame['lidar']['augmented_points'], raw_result)
print(fixed_result)

get_dashboard_image(
    diff_image,
    first_frame['lidar']['augmented_points'],
    apply_transformation_to_cloud(second_frame['lidar']['augmented_points'], fixed_result),
)

In [ ]:
# print(f'fix {first_i} - {second_i}')
# for field_i, k in enumerate(('x', 'y', 'teta')):
#     second_frame['tracker']['current_pose'][k] = first_frame['tracker']['current_pose'][k] + fixed_result[field_i]

In [ ]:
first_i = None
second_i = None
for i in tqdm(range(len(track))):
    if not track[i]['lidar']['augmented_points'] or not track[i]['tracker']['current_pose']:
        continue

    if first_i is None:
        first_i = i
        continue

    second_i = i
    first_frame = track[first_i]
    second_frame = track[second_i]

    raw_result = [
        second_frame['tracker']['current_pose'][k] - first_frame['tracker']['current_pose'][k]
        for field_i, k in enumerate(['x', 'y', 'teta'])
    ]
    if raw_result[0] == 0. and raw_result[1] == 0.:
        continue
    # print(f'({first_i}, {second_i})')
    fixed_result = fix_pose(second_frame['lidar']['augmented_points'], first_frame['lidar']['augmented_points'], raw_result)
    if not fixed_result:
        print(f'>_> ({first_i}, {second_i})')
        if first_i not in {126}:
            raise ValueError((first_i, second_i))
    # else:
    #     print(f'fix {first_i} - {second_i}')

    first_i = second_i